In [44]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.optimize import minimize
import geopy.distance
from math import radians, cos, sin, asin, sqrt
import pickle
import json

In [2]:
def preprocess_distances(cities, available_res, path='distances.pkl'):
    dist = np.zeros((n,k))
    for i in range(n):
        for j in range(k):
            dist[i,j] = geopy.distance.distance(cities.iloc[i][['lat','lng']].values, 
                                                available_res.iloc[j][['lat','lng']].values).km
    with open(path, 'wb') as out:
        pickle.dump(dist, out, pickle.HIGHEST_PROTOCOL)
    
def load_distances(path='distances.pkl'):
    with open('distances.pkl', 'rb') as inp:
        return pickle.load(inp)
        

In [82]:
earthquakes = [
    {'epicenter':(44.117512, 20.189369), 'magnitude': 6},
    {'epicenter':(43.886935, 20.556935), 'magnitude': 7},
    {'epicenter':(43.654546, 20.838094), 'magnitude': 7}
]

print(json.dumps(earthquakes))


[{"epicenter": [44.117512, 20.189369], "magnitude": 6}, {"epicenter": [43.886935, 20.556935], "magnitude": 7}, {"epicenter": [43.654546, 20.838094], "magnitude": 7}]


In [84]:
cities_df.head(5)

,city,population,lat,lng,resources
0,Belgrade,1378682.0,44.8167,20.4667,2757.364
1,Novi Sad,380000.0,45.2644,19.8317,760.000
2,Niš,183164.0,43.3192,21.8961,366.328
3,Zemun,161596.0,44.8500,20.4000,323.192
4,Kragujevac,150623.0,44.0142,20.9394,301.246


In [76]:
ugrozeni = pd.DataFrame(np.array([cities_df.city, goal]).T)
ugrozeni.columns = ['city', 'ugrozenih']
ugrozeni.to_json()

'{"city":{"0":"Belgrade","1":"Novi Sad","2":"Ni\\u0161","3":"Zemun","4":"Kragujevac","5":"Subotica","6":"Valjevo","7":"Loznica","8":"Zrenjanin","9":"Pan\\u010devo"},"ugrozenih":{"0":2832,"1":2707,"2":1753,"3":3364,"4":4959,"5":1133,"6":4473,"7":3568,"8":805,"9":2699}}'

In [46]:
# Prepare data
cities_df = pd.read_csv('serbia_cities.csv')[['city', 'population', 'lat', 'lng']].iloc[0:17]
n = len(cities_df)
np.random.seed(0)
goal = np.random.randint(100, 5000, size=n)

available_res = pd.DataFrame(cities_df)#.iloc[0:10]
k = len(available_res)
available_res['resources'] = available_res['population'] / 500
available_res_df = available_res.drop(['population'], axis=1)
resources = available_res_df['resources'].astype(np.int64)

preprocess_distances(cities_df, available_res)
dist = load_distances()

cities_df.head()

,city,population,lat,lng,resources
0,Belgrade,1378682.0,44.8167,20.4667,2757.364
1,Novi Sad,380000.0,45.2644,19.8317,760.000
2,Niš,183164.0,43.3192,21.8961,366.328
3,Zemun,161596.0,44.8500,20.4000,323.192
4,Kragujevac,150623.0,44.0142,20.9394,301.246


In [47]:
dist.shape

(10, 10)

In [59]:
available_res[['city','resources']].head()

,city,resources
0,Belgrade,2757.364
1,Novi Sad,760.000
2,Niš,366.328
3,Zemun,323.192
4,Kragujevac,301.246


In [58]:
available_res[['city','resources']].to_json()

'{"city":{"0":"Belgrade","1":"Novi Sad","2":"Ni\\u0161","3":"Zemun","4":"Kragujevac","5":"Subotica","6":"Valjevo","7":"Loznica","8":"Zrenjanin","9":"Pan\\u010devo"},"resources":{"0":2757.364,"1":760.0,"2":366.328,"3":323.192,"4":301.246,"5":211.362,"6":180.624,"7":172.826,"8":153.022,"9":152.406}}'

In [49]:
%%time
from scipy.optimize import minimize

# Utilities
def toVector(x): # n x k
    assert x.shape == (n, k)
    return x.flatten()

def toMat(vec):
    assert vec.shape == (n*k,)
    return vec.reshape(n,k)

# Cost function
alpha = 0.1
def cost_fun(x):
    x = toMat(x)
    cost1 = np.sum((np.sum(x, axis=1) - goal)**2)
    cost2 = np.sum(dist * x)
    return cost1 + alpha*cost2

# Constraints: k
def get_constraint(j):
    def constraint(x):
        x = toMat(x)
        return resources[j] - np.sum(x[:,j])
    return constraint

constraints = [{'type': 'ineq', 'fun':get_constraint(j)} 
                  for j in range(k)]

# Bounds
bnds = []
for i in range(n):
    for j in range(k):
        bnds.append((0, resources[j]))
print('Bounds:', len(bnds))

# initial guess
x0 = np.zeros((n, k))
#x0[0] = available_res

# show initial objective
print('Initial Objective: ' + str(cost_fun(toVector(x0))))

# optimize
solution = minimize(cost_fun, toVector(x0), bounds=(bnds), constraints=constraints)
x = np.array(solution.x).astype(np.int64)


# show final objective
print('Final Objective: ' + str(cost_fun(x)))

print('Used resources:', sum(x))

# print solution
print('Solution')
#print(x)

Bounds: 100
Initial Objective: 96283927.0
Final Objective: 57814278.93679594
Used resources: 5384
Solution
CPU times: user 2.6 s, sys: 3.98 ms, total: 2.61 s
Wall time: 2.61 s


In [50]:
print(goal)
print(sum(goal))

[2832 2707 1753 3364 4959 1133 4473 3568  805 2699]
28293


In [51]:
print(availaresources)

0    2757
1     760
2     366
3     323
4     301
5     211
6     180
7     172
8     153
9     152
Name: resources, dtype: int64


In [80]:
final_dist

city,Belgrade,Novi Sad,Niš,Zemun,Kragujevac,Subotica,Valjevo,Loznica,Zrenjanin,Pančevo
city,,,,,,,,,,
Belgrade,42,0,0,331,1129,0,1045,225,0,0
Novi Sad,5,0,0,55,199,0,276,228,0,0
Niš,0,0,0,0,365,0,0,0,0,0
Zemun,16,0,0,110,98,0,95,3,0,0
Kragujevac,0,0,0,0,300,0,0,0,0,0
Subotica,0,0,0,0,0,0,54,155,0,0
Valjevo,0,0,0,0,0,0,179,0,0,0
Loznica,0,0,0,0,0,0,0,171,0,0
Zrenjanin,18,0,0,65,5,0,42,23,0,0


In [79]:
final_dist = pd.DataFrame(toMat(x).T)
final_dist.columns = cities_df['city']
final_dist.index = cities_df['city']
print(final_dist.T.to_json())

{"Belgrade":{"Belgrade":42,"Novi Sad":0,"Ni\u0161":0,"Zemun":331,"Kragujevac":1129,"Subotica":0,"Valjevo":1045,"Loznica":225,"Zrenjanin":0,"Pan\u010devo":0},"Novi Sad":{"Belgrade":5,"Novi Sad":0,"Ni\u0161":0,"Zemun":55,"Kragujevac":199,"Subotica":0,"Valjevo":276,"Loznica":228,"Zrenjanin":0,"Pan\u010devo":0},"Ni\u0161":{"Belgrade":0,"Novi Sad":0,"Ni\u0161":0,"Zemun":0,"Kragujevac":365,"Subotica":0,"Valjevo":0,"Loznica":0,"Zrenjanin":0,"Pan\u010devo":0},"Zemun":{"Belgrade":16,"Novi Sad":0,"Ni\u0161":0,"Zemun":110,"Kragujevac":98,"Subotica":0,"Valjevo":95,"Loznica":3,"Zrenjanin":0,"Pan\u010devo":0},"Kragujevac":{"Belgrade":0,"Novi Sad":0,"Ni\u0161":0,"Zemun":0,"Kragujevac":300,"Subotica":0,"Valjevo":0,"Loznica":0,"Zrenjanin":0,"Pan\u010devo":0},"Subotica":{"Belgrade":0,"Novi Sad":0,"Ni\u0161":0,"Zemun":0,"Kragujevac":0,"Subotica":0,"Valjevo":54,"Loznica":155,"Zrenjanin":0,"Pan\u010devo":0},"Valjevo":{"Belgrade":0,"Novi Sad":0,"Ni\u0161":0,"Zemun":0,"Kragujevac":0,"Subotica":0,"Valjevo":17

In [53]:
sum(resources)

5375

In [9]:
%%time
np.array([[1,2,3],[4,5,6]]).flatten()

CPU times: user 56 µs, sys: 0 ns, total: 56 µs
Wall time: 60.6 µs


array([1, 2, 3, 4, 5, 6])